<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/dev/LangGraph/QnAagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from openai import OpenAI
from google.colab import userdata

# Set your OpenAI API key (from https://platform.openai.com/api-keys)
api_key=userdata.get("OPENAI_EDU_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # optional but recommended

# --- Setup the LLM ---
model = ChatOpenAI(
    model="gpt-4o",        # or "gpt-4o-mini", "gpt-4-turbo", etc.
    temperature=0.7        # optional, controls randomness
)

# --- Setup the Embedding model ---
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

In [ ]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
#print(get_laptop_price("alpha"))
#print(get_laptop_price("testing"))

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


In [ ]:
!pip install --upgrade --quiet  langgraph
!pip install --upgrade --quiet  pypdf
!pip install --upgrade --quiet  langchain-chroma

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
#print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [ ]:
# Example of how to invoke the agent
# Replace the question with your query about laptops
response = product_QnA_agent.invoke(
    {"messages": [HumanMessage(content="What is the price of AlphaBook Pro?")]},
    config={"configurable": {"thread_id": "some-thread-id"}} # Replace with a unique thread_id for each conversation
)

# Print the response from the agent
print(response)

{'messages': [HumanMessage(content='What is the price of AlphaBook Pro?', additional_kwargs={}, response_metadata={}, id='c5ad54a3-ca64-49bc-9b52-a21adeb6abc9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BvBwrswo07ZvYvQeRHVUotne', 'function': {'arguments': '{"laptop_name":"AlphaBook Pro"}', 'name': 'get_laptop_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 159, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_1827dd0c55', 'id': 'chatcmpl-CO7KwSdL5W87g40f59XPyGpu66MvM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ba246df9-4674-404f-a697-944b4526f71d-0', tool_calls=[{'name': 'get_laptop_price', 'args': {'lap

In [ ]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_cE1LTtb9UJDzIVfxig1n5jbU)
 Call ID: call_cE1LTtb9UJDzIVfxig1n5jbU
  Args:
    query: GammaAir
  get_laptop_price (call_jOucP9q0JBUT14BDJckvNDta)
 Call ID: call_jOucP9q0JBUT14BDJckvNDta
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X is a laptop that combines the following features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, perfect for users who need high performance in a portable design

The price for the GammaAir X is $1,399.


In [ ]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with that. Are there specific features or specifications you're looking for in a laptop? For example, CPU type, memory size, storage capacity, or any particular design features? Let me know your preferences so I can provide more tailored recommendations.
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here is a list of available laptops:

1. **AlphaBook Pro**
   - 12th Gen Intel i7 processor, 16GB DDR4 RAM, 1TB SSD.

2. **GammaAir X**
   - AMD Ryzen 7 processor, 32GB DDR4 RAM, 512GB NVMe SSD.

3. **SpectraBook S**
   - Intel Core i9 processor, 64GB RAM, 2TB SSD.

4. **OmegaPro G17**
   - Ryzen 9 5900HX CPU, 32GB RAM, 1TB SSD.

5. **NanoEdge Flex**

Let me know if you need more details about any of these laptops or if you want to know the

In [1]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")

NameError: name 'uuid' is not defined

In [5]:
import pandas as pd

#Load the laptop product orders CSV into a Pandas dataframe.
product_orders_df = pd.read_csv("Laptop Orders.csv")
print(product_orders_df)

   Order ID Product Ordered  Quantity Ordered Delivery Date
0  ORD-8276   SpectraBook S                 3    2024-10-16
1  ORD-6948    OmegaPro G17                 3    2024-10-25
2  ORD-7311   NanoEdge Flex                 2    2024-10-19
3  ORD-4633    OmegaPro G17                 2    2024-10-15
4  ORD-2050      GammaAir X                 2    2024-10-26


In [13]:
!pip install --upgrade --quiet langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 15.1 MB/s eta 0:00:00


In [9]:
!pip install --upgrade --quiet langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 13.9 MB/s eta 0:00:00


In [7]:
from langchain_core.tools import tool

@tool
def get_order_details(order_id:str) -> str :
    """
    This function returns details about a laptop order, given an order ID
    It performs an exact match between the input order id and available order ids
    If a match is found, it returns products (laptops) ordered, quantity ordered and delivery date.
    If there is NO match found, it returns -1
    """
    #Filter Dataframe for order ID
    match_order_df = product_orders_df[
                        product_orders_df["Order ID"] == order_id ]

    #Check if a record was found, if not return -1
    if len(match_order_df) == 0 :
        return -1
    else:
        return match_order_df.iloc[0].to_dict()

#Test the tool. Before running the test, comment the @tool annotation
#print(get_order_details("ORD-6948"))
#print(get_order_details("ORD-9999"))

@tool
def update_quantity(order_id:str, new_quantity:int) -> bool :
    """
    This function updates the quantity of products ( laptops ) ordered for a given order Id.
    It there are no matching orders, it returns False.
    """
    #Find if matching record exists
    match_order_df = product_orders_df[
                        product_orders_df["Order ID"] == order_id ]

    #Check if a record was found, if not return -1
    if len(match_order_df) == 0 :
        return -1
    else:
        product_orders_df.loc[
            product_orders_df["Order ID"] == order_id,
                "Quantity Ordered"] = new_quantity
        return True

#Test the tool. Before running the test, comment the @tool annotation
#print(get_order_details("ORD-6948"))
#print(update_quantity("ORD-6948", 1))
#print(get_order_details("ORD-6948"))
#print(update_quantity("ORD-9999",2))
#print(product_orders_df)

In [24]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from IPython.display import Image
import json
from langchain_core.runnables.graph import MermaidDrawMethod

#An Agent State class that keep state of the agent while it answers a query
class OrdersAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

#-----------------------------------------------------------------------------
#An agent class that manages all agentic interactions
class OrdersAgent:

    #Setup the agent graph, tools and memory
    def __init__(self, model, tools, system_prompt, debug):

        self.system_prompt=system_prompt
        self.debug=debug

        #Setup the graph for the agent manually
        agent_graph=StateGraph(OrdersAgentState)
        agent_graph.add_node("orders_llm",self.call_llm)
        agent_graph.add_node("orders_tools",self.call_tools)
        agent_graph.add_conditional_edges(
            "orders_llm",
            self.is_tool_call,
            {True: "orders_tools", False: END }
        )
        agent_graph.add_edge("orders_tools","orders_llm")
        #Set where there graph starts
        agent_graph.set_entry_point("orders_llm")

        #Add chat memory
        self.memory=MemorySaver()
        #compile the graph
        self.agent_graph = agent_graph.compile(checkpointer=self.memory)

        #Setup tools
        self.tools = { tool.name : tool for tool in tools }
        if self.debug:
            print("\nTools loaded :", self.tools)

        #attach tools to model
        self.model=model.bind_tools(tools)


    #Call the LLM with the messages to get next action/result
    def call_llm(self, state:OrdersAgentState):

        messages=state["messages"]

        #If system prompt exists, add to messages in the front
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages

        #invoke the model with the message history
        result = self.model.invoke(messages)
        if self.debug:
            print(f"\nLLM Returned : {result}")
        #Return the LLM output
        return { "messages":[result] }


    #Check if the next action is a tool call.
    def is_tool_call(self, state:OrdersAgentState):
        last_message = state["messages"][-1]
        #print("Last result from LLM : ", last_message)
        #If tool action is requested
        if len(last_message.tool_calls) > 0 :
            return True
        else:
            return False

    #Execute the tool requested with the given parameters
    def call_tools(self, state:OrdersAgentState):
        #Get last message
        tool_calls = state["messages"][-1].tool_calls
        results=[]

        #Multiple tool calls may be requested. Execute one by one
        for tool in tool_calls:
            #Handle tool missing error
            if not tool["name"] in self.tools:
                print(f"Unknown tool name {tool}")
                result = "Invalid tool found. Please retry"
            else:
                #Call the tool and collect results
                result=self.tools[tool["name"]].invoke(tool["args"])

            #append results to the list of tool results
            results.append(ToolMessage(tool_call_id=tool['id'],
                                       name=tool['name'],
                                       content=str(result)))

            if self.debug:
                print(f"\nTools returned {results}")
            #return tool results
            return { "messages" : results }

#-----------------------------------------------------------------------------
#Setup the custom orders agent

#Note that this is a string, since the model init only accepts a string.
system_prompt = """
    You are professional chatbot that manages orders for laptops sold by our company.
    The tools allow for retrieving order details as well as update order quantity.
    Do NOT reveal information about other orders than the one requested.
    You will handle small talk and greetings by producing professional responses.
    """

#Create the custom orders agent
orders_agent = OrdersAgent(model,
                           [get_order_details, update_quantity],
                           system_prompt,
                           debug=False)

#Visualize the Agent
#Image(orders_agent.agent_graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER))

In [25]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "How are you doing?",
    "Please show me the details of the order ORD-7311",
    "Can you add one more of that laptop to the order? ",
    "Can you show me the details again ? ",
    "What about order ORD-9999 ?",
    "Bye"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = orders_agent.agent_graph.invoke(user_message,config=config)
    #Print the response
    print(f"\nAGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : How are you doing?

AGENT : I'm here and ready to assist you with your laptop orders. How can I help you today?
----------------------------------------
USER : Please show me the details of the order ORD-7311

AGENT : Here are the details for order **ORD-7311**:

- **Product Ordered:** NanoEdge Flex
- **Quantity Ordered:** 2
- **Delivery Date:** 2024-10-19

If you need any further assistance, feel free to ask!
----------------------------------------
USER : Can you add one more of that laptop to the order? 

AGENT : The quantity for order **ORD-7311** has been successfully updated to 3. If you need any more help, feel free to let me know!
----------------------------------------
USER : Can you show me the details again ? 

AGENT : Here are the updated details for order **ORD-7311**:

- **Product Ordered:** NanoEdge Flex
- **Quantity Ordered:** 3
- **Delivery Date:** 2024-10-19

If there's anything else you need, feel free to ask!
--------